In [3]:
%reload_ext autoreload
%autoreload 1
import torch 
import sys
sys.path.append('..')
from torch import nn 
from torch.nn import functional as F
from torch import optim
from utils.loader import load
# from models.vince_models import Net2B

In [4]:
# load the dataset
train_input,train_target, train_classes, test_input, test_target, test_classes = load()

mean, std = train_input.mean(), train_input.std()

train_input = train_input.sub_(mean).div_(std)
test_input = test_input.sub_(mean).div_(std)

In [8]:
# architectures 

class LeNet_WS_sequential(nn.Module):
    """
    Weight sharing 
    
    """
    def __init__(self, nb_hidden):
        super(LeNet_WS_sequential, self).__init__()
        
        # convolutional weights for digit reocgnition shared for each image
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(256, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 10)
        # fully connected layers 
        self.fc3 = nn.Linear(20, 60)
        self.fc4 = nn.Linear(60, 90)
        self.fc5 = nn.Linear(90, 2)
        
    def forward(self, input_):        
        
        # split the 2-channel input into two 14*14 images
        x = input_[:, 0, :, :].view(-1, 1, 14, 14)
        y = input_[:, 1, :, :].view(-1, 1, 14, 14)
        
        # forward pass for the first image 
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=2, stride=2))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = F.relu(self.fc1(x.view(-1, 256)))
        x = self.fc2(x)
        # forward pass for the second image
        y = F.relu(F.max_pool2d(self.conv1(y), kernel_size=2, stride=2))
        y = F.relu(F.max_pool2d(self.conv2(y), kernel_size=2, stride=2))
        y = F.relu(self.fc1(y.view(-1, 256)))
        y = self.fc2(y)
        
        # concatenate layers 
        z = torch.cat([x, y], 1)
        
        z = F.relu(self.fc3(z))
        z = F.relu(self.fc4(z))
        z = self.fc5(z)
        
        return  z

    


In [9]:
##### train function ######

def train_WS (model, train_input, train_target, train_classes, optimizer = optim.SGD,
                criterion = nn.CrossEntropyLoss(), n_epochs=50, mini_batch_size=100, eta=1e-1, lambda_l2 = 0):
    
    """
    Train network with simple weight sharing
    
    """

    model.train()
    optimizer = optimizer(model.parameters(), lr = eta)
    
    for e in range(n_epochs):
        epoch_loss = 0
        
        for input_, target_, classes_ in (zip(train_input.split(mini_batch_size), train_target.split(mini_batch_size), 
                                              train_classes.split(mini_batch_size))):

      
            
            out_loss  = criterion(model(input_), target_)
            
            epoch_loss += out_loss
            
            if lambda_l2 != 0:
                for p in model.parameters():
                    epoch_loss += lambda_l2 * p.pow(2).sum() # add an l2 penalty term to the loss 
            
            optimizer.zero_grad()
            out_loss.backward()
            optimizer.step()
            
        print('Train Epoch: {}  | Loss {:.6f}'.format(
                e, epoch_loss.item()))
        
#########################################################################################################################
#########################################################################################################################

### test function  ###

def test_WS (model, test_input, test_target, mini_batch_size=100, criterion = nn.CrossEntropyLoss()):
    
    """
    Test function to calculate prediction accuracy of a binary cnn classifier
    
    """
    
    model.eval()
    test_loss = 0
    nb_errors=0
    
    with torch.no_grad():
        
        for input_, target_ in zip(test_input.split(mini_batch_size), test_target.split(mini_batch_size)):
            
            output = model(input_) 
            batch_loss = criterion(output, target_)
            test_loss += batch_loss
            
            _, predicted_classes = output.max(1)
            for k in range(mini_batch_size):
                if target_[k] != predicted_classes[k]:
                    nb_errors = nb_errors + 1
                                   
             
        print('\nTest set | Loss: {:.4f} | Accuracy: {:.0f}% | # misclassified : {}/{}\n'.format(
        test_loss.item(), 100 * (len(test_target)-nb_errors)/len(test_target), nb_errors, len(test_target)))
        
        print(nb_errors)
        
    

In [11]:
model = LeNet_WS_sequential(500)
train_WS(model, train_input, train_target, train_classes)
test_WS(model, test_input, test_target)

Train Epoch: 0  | Loss 6.969314
Train Epoch: 1  | Loss 6.889643
Train Epoch: 2  | Loss 6.871070
Train Epoch: 3  | Loss 6.859251
Train Epoch: 4  | Loss 6.844707
Train Epoch: 5  | Loss 6.823288
Train Epoch: 6  | Loss 6.789163
Train Epoch: 7  | Loss 6.730149
Train Epoch: 8  | Loss 6.621634
Train Epoch: 9  | Loss 6.401676
Train Epoch: 10  | Loss 5.939711
Train Epoch: 11  | Loss 5.126228
Train Epoch: 12  | Loss 4.446487
Train Epoch: 13  | Loss 4.001048
Train Epoch: 14  | Loss 3.613821
Train Epoch: 15  | Loss 3.281789
Train Epoch: 16  | Loss 2.969346
Train Epoch: 17  | Loss 2.703670
Train Epoch: 18  | Loss 2.434501
Train Epoch: 19  | Loss 2.139358
Train Epoch: 20  | Loss 1.805368
Train Epoch: 21  | Loss 1.755644
Train Epoch: 22  | Loss 1.537876
Train Epoch: 23  | Loss 1.448249
Train Epoch: 24  | Loss 1.162417
Train Epoch: 25  | Loss 1.481576
Train Epoch: 26  | Loss 0.651011
Train Epoch: 27  | Loss 0.402218
Train Epoch: 28  | Loss 0.257624
Train Epoch: 29  | Loss 0.149405
Train Epoch: 30  | L

In [12]:
trainloader = torch.utils.data.DataLoader(train_input, batch_size=100, shuffle=False)

In [16]:
trainloader

TypeError: 'DataLoader' object is not subscriptable